In [1]:
import numpy as np
from skimage.transform import resize
translator = dict()
retranslator = np.empty([1000])
temp = 0

def preparing_img(img):
    return resize(img, (64, 64), mode = 'reflect')

def preparing_label(label):
    global temp
    if (translator.get(label) == None):
        translator[label] = temp
        retranslator[temp] = label
        temp += 1
    label = translator[label]
    res = np.zeros([1000])
    res[label] = 1
    return res
        
def preparing_test(test):
    res = np.empty([83247, 64, 64])
    for i in range(len(test)):
        res[i] = resize(test[i], [64, 64], mode = 'reflect')
        test[i] = np.empty([0])
    test = np.empty([0])
    return res

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [2]:
import numpy as np
for train_num in range(1, 5):
    data_name = 'data/train-' + str(train_num) + '.npy'
    data = np.load(data_name)
    imgs = data.T[0]
    labels = data.T[1]
    data = np.empty([0])
    np.save('data/imgs-' + str(train_num) + '.npy', np.array([preparing_img(img) for img in imgs]))
    imgs = np.empty([0])
    np.save('data/labels-' + str(train_num) + '.npy', np.array([preparing_label(label) for label in labels]))
    labels = np.empty([0])
    print('.')
    
np.save('data/retranslator.npy', retranslator)
test = np.load('data/test.npy')
np.save('data/pretest.npy', np.array([preparing_img(img) for img in test]))
test = np.empty([0])

.
.
.
.


In [2]:
def next_batch(num, data, labels):
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    '''for i in range(len(labels_shuffle)):
        place = labels_shuffle[i]
        labels_shuffle[i] = np.zeros(1000)
        labels_shuffle[i][place] = 1'''
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [6]:
import tensorflow as tf
l = 0.1
def leaky_relu(features):
    return (1 - l) * tf.nn.relu(features)

In [ ]:
from math import exp
l1, l2 = exp(-10), exp(-10)
def gradient_descent(shapes, nonlinearity):
    tf.get_variable_scope().reuse_variables()
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.float32, shape=[None, 4320])
    y_ = tf.placeholder(tf.float32, shape=[None, 1000])
    
    list_of_layers = [x]
    for i in range(1, len(shapes) - 1):
        list_of_layers.append(
            tf.layers.dense(
                    inputs=list_of_layers[i-1],
                    units=shapes[i],
                    activation=nonlinearity,
                    use_bias=True,
                    kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1, l2)
                ))

    logits = tf.layers.dense(
                    inputs=list_of_layers[-1],
                    units=shapes[-1],
                    use_bias=True,
                    kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(l1, l2)
                )
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits = logits)) + tf.losses.get_regularization_loss()
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    
    prediction = tf.argmax(logits,1)
    correct_prediction = tf.equal(prediction, tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for train_num in range(1, 5):
            data_name = 'data/train-' + str(train_num) + '.npy'
            data = np.load(data_name)
            preparing(data)
            for i in range(12000):
                batch = next_batch(100, data.T[0], data.T[1])
                train_step.run(feed_dict={x: batch[0], y_: batch[1]})
                if (i % 500 == 0):
                    print('.', end = '')
            print('#')
            batch = next_batch(3000, data.T[0], data.T[1])
            data = np.empty([0])
            print("accuracy:", accuracy.eval(feed_dict={x: batch[0], y_: batch[1]}))
            batch = np.empty([0])
        test = np.load('data/test.npy')
        test.reshape(83247, 1)
        test = preparing_test(test)
        print("test ready, now session")
        return sess.run(prediction, feed_dict = {x: test})

In [4]:
import pandas as pd
pd.__version__

'0.21.0'

In [ ]:
from time import time
import pandas as pd
import numpy as np

start = time()
res = gradient_descent([4096, 2548, 1774, 1000], leaky_relu)
print("time:", time() - start)

In [ ]:
df = pd.DataFrame(columns=['Id', 'Category'])
for i in range(len(res)):
    df.loc[i] = [i + 1, retranslator(res[i])]

In [ ]:
df.to_csv('out.csv')

ANOTHER NEURAL NETWORK JUST COPY OF DEEP TUTORIAL

In [7]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [8]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
import tensorflow as tf

def gradient_descent_perfecto():
    tf.get_variable_scope().reuse_variables()
    tf.reset_default_graph()
    
    x = tf.placeholder(tf.float32, shape=[None, 72, 60])
    y_ = tf.placeholder(tf.float32, shape=[None, 1000])
    
    #first conv
    W_conv1 = weight_variable([5, 5, 1, 32])
    b_conv1 = bias_variable([32])
    
    x_image = tf.reshape(x, [-1, 72, 60, 1])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)
    
    # second conv
    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2)
    
    W_fc1 = weight_variable([18 * 15 * 64, 6144])
    b_fc1 = bias_variable([6144])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 18 * 15 * 64])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
    W_fc2 = weight_variable([6144, 1000])
    b_fc2 = bias_variable([1000])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
    
    cross_entropy = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    prediction = tf.argmax(y_conv, 1)
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for train_num in range(1, 5):
            data_name = 'data/train-' + str(train_num) + '.npy'
            data = np.load(data_name)
            preparing(data)
            for i in range(20000):
                batch = next_batch(100, data.T[0], data.T[1])
                train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
                if (i % 500 == 0):
                    print('.', end = '')
            print('#')
            batch = next_batch(3000, data.T[0], data.T[1])
            data = np.empty([0])
            print("accuracy:", accuracy.eval(feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.0}))
            batch = np.empty([0])
        test = np.load('data/test.npy')
        test.reshape(83247, 1)
        test = preparing_test(test)
        print("test ready, now session")
        return sess.run(prediction, feed_dict = {x: test, keep_prob: 1.0})

In [10]:
from time import time
import pandas as pd
import numpy as np

start = time()
res = gradient_descent_perfecto()
print("time:", time() - start)

ResourceExhaustedError: OOM when allocating tensor with shape[17280,6144]
	 [[Node: Variable_4/Adam_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_4"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable_4/Adam_1, Variable_4/Adam/Initializer/zeros)]]

Caused by op 'Variable_4/Adam_1/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", lin

In [ ]:
df = pd.DataFrame(columns=['Id', 'Category'])
for i in range(len(res)):
    df.loc[i] = [i + 1, retranslator(res[i])]

In [ ]:
df.to_csv('out.csv')

In [1]:
from keras.preprocessing.image import ImageDataGenerator
def IMDG(x_train):
    datagen = ImageDataGenerator(
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.03,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.03)  # randomly shift images vertically (fraction of total height)
    return datagen

Using TensorFlow backend.


In [2]:
def saver(path, res):
    retranslator = np.load('data/retranslator.npy')
    with open(path, 'w') as out:
        print('Id,Category', file=out)
        for i in range(len(res)):
            print('{i},{res}'.format(i = i + 1, res=int(retranslator[np.argmax(res[i])])), file=out)

In [2]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

# Generate dummy data

x = Input(shape=(64, 64, 1))

conv = Conv2D(64, kernel_size=(7, 7), padding = 'same', strides = 2)(x)
m_pool = MaxPooling2D(pool_size=(2, 2))(conv)

conv_1 = Conv2D(64, kernel_size=(3, 3), padding = 'same')(m_pool)
batch_n_1 = BatchNormalization(axis = 3)(conv_1)
leaky_1 = LeakyReLU()(batch_n_1)
drop_1 = Dropout(0.25)(leaky_1)
conv_1_2 = Conv2D(64, kernel_size=(3, 3), padding = 'same')(drop_1)
res_1 = keras.layers.add([conv_1_2, m_pool])
batch_n_1_2 = BatchNormalization(axis = 3)(res_1)
leaky_1_2 = LeakyReLU()(batch_n_1_2)
drop_1_2 = Dropout(0.25)(leaky_1_2)

conv_2 = Conv2D(64, kernel_size=(3, 3), padding = 'same')(drop_1_2)
batch_n_2 = BatchNormalization(axis = 3)(conv_2)
leaky_2 = LeakyReLU()(batch_n_2)
drop_2 = Dropout(0.25)(leaky_2)
conv_2_2 = Conv2D(64, kernel_size=(3, 3), padding = 'same')(drop_2)
res_2 = keras.layers.add([conv_2_2, drop_1_2])
batch_n_2_2 = BatchNormalization(axis = 3)(res_2)
leaky_2_2 = LeakyReLU()(batch_n_2_2)
drop_2_2 = Dropout(0.25)(leaky_2_2)

conv_3 = Conv2D(64, kernel_size=(3, 3), padding = 'same')(drop_2_2)
batch_n_3 = BatchNormalization(axis = 3)(conv_3)
leaky_3 = LeakyReLU()(batch_n_3)
drop_3 = Dropout(0.25)(leaky_3)
conv_3_2 = Conv2D(64, kernel_size=(3, 3), padding = 'same')(drop_3)
res_3 = keras.layers.add([conv_3_2, drop_2_2])
batch_n_3_2 = BatchNormalization(axis = 3)(res_3)
leaky_3_2 = LeakyReLU()(batch_n_3_2)
drop_3_2 = Dropout(0.25)(leaky_3_2)

conv_128 = Conv2D(128, kernel_size=(3, 3), padding = 'same', strides = 2)(drop_3_2)

conv_4 = Conv2D(128, kernel_size=(3, 3), padding = 'same')(conv_128)
batch_n_4 = BatchNormalization(axis = 3)(conv_4)
leaky_4 = LeakyReLU()(batch_n_4)
drop_4 = Dropout(0.25)(leaky_4)
conv_4_2 = Conv2D(128, kernel_size=(3, 3), padding = 'same')(drop_4)
res_4 = keras.layers.add([conv_4_2, conv_128])
batch_n_4_2 = BatchNormalization(axis = 3)(res_4)
leaky_4_2 = LeakyReLU()(batch_n_4_2)
drop_4_2 = Dropout(0.25)(leaky_4_2)

conv_5 = Conv2D(128, kernel_size=(3, 3), padding = 'same')(drop_4_2)
batch_n_5 = BatchNormalization(axis = 3)(conv_5)
leaky_5 = LeakyReLU()(batch_n_5)
drop_5 = Dropout(0.25)(leaky_5)
conv_5_2 = Conv2D(128, kernel_size=(3, 3), padding = 'same')(drop_5)
res_5 = keras.layers.add([conv_5_2, drop_4_2])
batch_n_5_2 = BatchNormalization(axis = 3)(res_5)
leaky_5_2 = LeakyReLU()(batch_n_5_2)
drop_5_2 = Dropout(0.25)(leaky_5_2)

conv_6 = Conv2D(128, kernel_size=(3, 3), padding = 'same')(drop_5_2)
batch_n_6 = BatchNormalization(axis = 3)(conv_6)
leaky_6 = LeakyReLU()(batch_n_6)
drop_6 = Dropout(0.25)(leaky_6)
conv_6_2 = Conv2D(128, kernel_size=(3, 3), padding = 'same')(drop_6)
res_6 = keras.layers.add([conv_6_2, drop_5_2])
batch_n_6_2 = BatchNormalization(axis = 3)(res_6)
leaky_6_2 = LeakyReLU()(batch_n_6_2)
drop_6_2 = Dropout(0.25)(leaky_6_2)

conv_256 = Conv2D(256, kernel_size=(3, 3), padding = 'same', strides = 2)(drop_6_2)

conv_7 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(conv_256)
batch_n_7 = BatchNormalization(axis = 3)(conv_7)
leaky_7 = LeakyReLU()(batch_n_7)
drop_7 = Dropout(0.25)(leaky_7)
conv_7_2 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_7)
res_7 = keras.layers.add([conv_7_2, conv_256])
batch_n_7_2 = BatchNormalization(axis = 3)(res_7)
leaky_7_2 = LeakyReLU()(batch_n_7_2)
drop_7_2 = Dropout(0.25)(leaky_7_2)

conv_8 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_7_2)
batch_n_8 = BatchNormalization(axis = 3)(conv_8)
leaky_8 = LeakyReLU()(batch_n_8)
drop_8 = Dropout(0.25)(leaky_8)
conv_8_2 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_8)
res_8 = keras.layers.add([conv_8_2, drop_7_2])
batch_n_8_2 = BatchNormalization(axis = 3)(res_8)
leaky_8_2 = LeakyReLU()(batch_n_8_2)
drop_8_2 = Dropout(0.25)(leaky_8_2)

conv_9 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_8_2)
batch_n_9 = BatchNormalization(axis = 3)(conv_9)
leaky_9 = LeakyReLU()(batch_n_9)
drop_9 = Dropout(0.25)(leaky_9)
conv_9_2 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_9)
res_9 = keras.layers.add([conv_9_2, drop_8_2])
batch_n_9_2 = BatchNormalization(axis = 3)(res_9)
leaky_9_2 = LeakyReLU()(batch_n_9_2)
drop_9_2 = Dropout(0.25)(leaky_9_2)

conv_10 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_9_2)
batch_n_10 = BatchNormalization(axis = 3)(conv_10)
leaky_10 = LeakyReLU()(batch_n_10)
drop_10 = Dropout(0.25)(leaky_10)
conv_10_2 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_10)
res_10 = keras.layers.add([conv_10_2, drop_9_2])
batch_n_10_2 = BatchNormalization(axis = 3)(res_10)
leaky_10_2 = LeakyReLU()(batch_n_10_2)
drop_10_2 = Dropout(0.25)(leaky_10_2)

conv_11 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_10_2)
batch_n_11 = BatchNormalization(axis = 3)(conv_11)
leaky_11 = LeakyReLU()(batch_n_11)
drop_11 = Dropout(0.25)(leaky_11)
conv_11_2 = Conv2D(256, kernel_size=(3, 3), padding = 'same')(drop_11)
res_11 = keras.layers.add([conv_11_2, drop_10_2])
batch_n_11_2 = BatchNormalization(axis = 3)(res_11)
leaky_11_2 = LeakyReLU()(batch_n_11_2)
drop_11_2 = Dropout(0.25)(leaky_11_2)

#conv_512 = Conv2D(512, kernel_size=(3, 3), padding = 'same', strides = 2)(batch_n_11_2)

apool = keras.layers.AveragePooling2D(pool_size=(2, 2), padding='same')(drop_11_2)

flat = Flatten()(apool)

dense = Dense(1000, activation = 'softmax')(flat)
model = Model(inputs=[x], outputs=[dense])

# initiate RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()
"""batch_size = [40, 80, 160, 320, 480]
for _ in range(len(batch_size)):
    nums = np.array([1, 2, 3, 4])
    np.random.shuffle(nums)
    for train_num in nums:
        print(_ + 1, 'из', 4)
        imgs = np.load('data/imgs-' + str(train_num) + '.npy')
        labels = np.load('data/labels-' + str(train_num) + '.npy')
        model.fit(np.reshape(imgs, [len(imgs), 64, 64, 1]), labels, batch_size = batch_size[_], epochs = 1, verbose = 1)
        imgs = np.empty([0])
        labels = np.empty([0])

    test = np.load('data/pretest.npy')
    res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
    test = np.empty([0])
    saver('out' + str(6 + _) + '.csv', res)
    res = np.empty([0])
    model.save('my_model.h5')"""
nums = np.array([1, 2, 3, 4])
#np.random.shuffle(nums)
for train_num in nums:
    print(train_num, 'from', len(nums))
    imgs = np.load('data/imgs-' + str(train_num) + '.npy')
    imgs = np.reshape(imgs, [len(imgs), 64, 64, 1])
    labels = np.load('data/labels-' + str(train_num) + '.npy')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.03,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.03)  # randomly shift images vertically (fraction of total height)
    datagen.fit(imgs)
    model.fit_generator(datagen.flow(imgs, labels, batch_size=69),
                        steps_per_epoch = (len(imgs) / 69), epochs=2, verbose=1)
    datagen = 0
    imgs = 0
    labels = 0
    if (train_num < 4):
        imgs = np.load('data/imgs-' + str(4) + '.npy')[:20000]
        imgs = np.reshape(imgs, [len(imgs), 64, 64, 1])
        labels = np.load('data/labels-' + str(4) + '.npy')[:20000]
        score = model.evaluate(imgs, labels, batch_size=100)
        print(score)
        imgs = 0
        labels = 0
    model.save('my_model.h5')
test = np.load('data/pretest.npy')
res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
test = np.empty([0])
saver('out' + str(6) + '.csv', res)
res = np.empty([0])

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 64, 64, 1)     0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 32, 32, 64)    3200        input_1[0][0]                    
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 16, 16, 64)    0           conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 16, 16, 64)    36928       max_pooling2d_1[0][0]            
___________________________________________________________________________________________

Epoch 1/2
1207/1206 [==============================] - 587s - loss: 4.5843 - acc: 0.1374   
Epoch 2/2
20000/20000 [==============================] - 37s    
[1.232869561612606, 0.67335000365972519]
2 from 4
Epoch 1/2
1207/1206 [==============================] - 558s - loss: 1.0439 - acc: 0.7171   
Epoch 2/2
20000/20000 [==============================] - 36s    
[3.8966691255569459, 0.28420000046491622]
3 from 4
Epoch 1/2
1207/1206 [==============================] - 574s - loss: 0.6129 - acc: 0.8254   
Epoch 2/2
20000/20000 [==============================] - 37s    
[0.53421889148652557, 0.85149999737739568]
4 from 4
Epoch 1/2
1207/1206 [==============================] - 558s - loss: 0.5225 - acc: 0.8508   
Epoch 2/2
1207/1206 [==============================] - 557s - loss: 0.4397 - acc: 0.8728   


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [1]:
from keras.models import load_model
model = load_model('my_model.h5')

Using TensorFlow backend.


In [4]:
batch_size = 240
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
nums = np.array([4, 2, 3, 1])
for train_num in nums:
    print(train_num, 'from', len(nums))
    imgs = np.load('data/imgs-' + str(train_num) + '.npy')
    imgs = np.reshape(imgs, [len(imgs), 64, 64, 1])
    labels = np.load('data/labels-' + str(train_num) + '.npy')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        rotation_range=4,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.03,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.03)  # randomly shift images vertically (fraction of total height)
    datagen.fit(imgs)
    model.fit_generator(datagen.flow(imgs, labels, batch_size=batch_size),
                        steps_per_epoch = (len(imgs) / batch_size), epochs=2, verbose=1)
    datagen = 0
    imgs = 0
    labels = 0
    model.save('my_model1.h5')
test = np.load('data/pretest.npy')
res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
test = np.empty([0])
saver('out' + str(7) + '.csv', res)
res = np.empty([0])

4 from 4
Epoch 1/2
347/346 [==============================] - 446s - loss: 0.1579 - acc: 0.9524   
Epoch 2/2
347/346 [==============================] - 443s - loss: 0.1296 - acc: 0.9597   
2 from 4
Epoch 1/2
347/346 [==============================] - 443s - loss: 0.1573 - acc: 0.9536   
Epoch 2/2
347/346 [==============================] - 447s - loss: 0.1217 - acc: 0.9631   
3 from 4
Epoch 1/2
347/346 [==============================] - 443s - loss: 0.1419 - acc: 0.9573   
Epoch 2/2
347/346 [==============================] - 444s - loss: 0.1140 - acc: 0.9651   
1 from 4
Epoch 1/2
347/346 [==============================] - 445s - loss: 0.1052 - acc: 0.9684   
Epoch 2/2
347/346 [==============================] - 447s - loss: 0.0838 - acc: 0.9747   


In [4]:
model.save('my_model.h5')

In [3]:
test = np.load('data/pretest.npy')
res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
test = np.empty([0])
saver('out6.csv', res)
res = np.empty([0])

In [4]:
imgs = np.empty([0])
labels = np.empty([0])

In [24]:
test = np.load('data/pretest.npy')
res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
test = np.empty([0])
def saver(path, res):
    retranslator = np.load('data/retranslator.npy')
    with open(path, 'w') as out:
        print('Id,Category', file=out)
        for i in range(len(res)):
            print('{i},{res}'.format(i = i + 1, res=int(retranslator[np.argmax(res[i])])), file=out)

In [25]:
import pandas as pd
df = pd.read_csv('out5.csv')
df.head()

,Id,Category
0,1,63955
1,2,64432
2,3,64709
3,4,64177
4,5,61881


In [23]:
res.shape

(83247, 4)

In [7]:
res = np.empty((83247, 0), dtype = np.int64)
for num in [1, 3, 5, 6]:
    df = pd.read_csv('out' + str(num) + '.csv')
    res_i = df['Category']
    res = np.hstack((res, res_i.reshape((83247, 1))))
print(res[100], np.bincount(res[100]).argmax())
path = 'out4.csv'
with open(path, 'w') as out:
    print('Id,Category', file=out)
    for i in range(len(res)):
        print('{i},{res}'.format(i= i + 1, res=np.bincount(res[i]).argmax()), file=out)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """


[64202 64202 64202 64202] 64202


In [6]:
import pandas as pd
df = pd.read_csv('out4.csv')
df.head()

,Id,Category
0,1,63955
1,2,64432
2,3,64709
3,4,64177
4,5,61881


In [31]:
print(res[103], np.bincount(res[103]).argmax())

[61623 61623 61623] 61623


In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

# Generate dummy data

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64, 64, 1)))
model.add(LeakyReLU())
model.add(Conv2D(32, (3, 3)))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(64, (3, 3)))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(LeakyReLU())
model.add(Conv2D(128, (3, 3)))
model.add(LeakyReLU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization(axis=3))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(2200))
model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Dense(1000))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
model.summary()
batch_size = [40, 80, 140, 200]
for _ in range(4):
    for train_num in range(1, 5):
        print(_ + 1, 'из', 3)
        imgs = np.load('data/imgs-' + str(train_num) + '.npy')
        labels = np.load('data/labels-' + str(train_num) + '.npy')
        model.fit(np.reshape(imgs, [len(imgs), 64, 64, 1]), labels, batch_size = batch_size[_], epochs = 1, verbose = 1)
        imgs = np.empty([0])
        labels = np.empty([0])
test = np.load('data/pretest.npy')
res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
test = np.empty([0])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 32)        9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 62, 62, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 31, 32)        128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 32)        0         
__________

In [3]:
for train_num in range(4, 0, -1):
    print(5, 'из', 3)
    imgs = np.load('data/imgs-' + str(train_num) + '.npy')
    labels = np.load('data/labels-' + str(train_num) + '.npy')
    model.fit(np.reshape(imgs, [len(imgs), 64, 64, 1]), labels, batch_size = 200, epochs = 1, verbose = 1)
    imgs = np.empty([0])
    labels = np.empty([0])

def saver(path, res):
    retranslator = np.load('data/retranslator.npy')
    with open(path, 'w') as out:
        print('Id,Category', file=out)
        for i in range(len(res)):
            print('{i},{res}'.format(i = i + 1, res=int(retranslator[np.argmax(res[i])])), file=out)

#res = np.array([retranslator[np.argmax(elem)] for elem in res])
saver('out7.csv', res)

5 из 3
Epoch 1/1
83247/83247 [==============================] - 439s - loss: 0.0549 - acc: 0.9829   
5 из 3
Epoch 1/1
83247/83247 [==============================] - 445s - loss: 0.0683 - acc: 0.9795   
5 из 3
Epoch 1/1
83247/83247 [==============================] - 451s - loss: 0.0812 - acc: 0.9755   
5 из 3
Epoch 1/1
83246/83246 [==============================] - 453s - loss: 0.0579 - acc: 0.9838   


In [4]:
test = np.load('data/pretest.npy')
res = model.predict(np.reshape(test, [len(test), 64, 64, 1]))
test = np.empty([0])

def saver(path, res):
    retranslator = np.load('data/retranslator.npy')
    with open(path, 'w') as out:
        print('Id,Category', file=out)
        for i in range(len(res)):
            print('{i},{res}'.format(i = i + 1, res=int(retranslator[np.argmax(res[i])])), file=out)

#res = np.array([retranslator[np.argmax(elem)] for elem in res])
saver('out7.csv', res)

In [2]:
x = Input(shape=(64, 64, 1))

conv_1 = Conv2D(32, kernel_size=(3, 3))(x)
batch_n_1 = BatchNormalization(axis = 3)(conv_1)
leaky_1 = LeakyReLU()(batch_n_1)
conv_1_2 = Conv2D(32, kernel_size=(3, 3))(leaky_1)
res_1 = keras.layers.concatenate([conv_1_2, x])
batch_n_1_2 = BatchNormalization(axis = 3)(res_1)

conv_2 = Conv2D(32, kernel_size=(3, 3))(batch_n_1_2)
batch_n_2 = BatchNormalization(axis = 3)(conv_2)
leaky_2 = LeakyReLU()(batch_n_2)
conv_2_2 = Conv2D(32, kernel_size=(3, 3))(leaky_2)
res_2 = keras.layers.concatenate([conv_2_2, batch_n_1_2])
batch_n_2_2 = BatchNormalization(axis = 3)(res_2)

conv_3 = Conv2D(32, kernel_size=(3, 3))(batch_n_2_2)
batch_n_3 = BatchNormalization(axis = 3)(conv_3)
leaky_3 = LeakyReLU()(batch_n_3)
conv_3_2 = Conv2D(32, kernel_size=(3, 3))(leaky_3)
res_3 = keras.layers.concatenate([conv_3_2, batch_n_2_2])
batch_n_3_2 = BatchNormalization(axis = 3)(res_3)

flat = Flatten()(batch_n_3_2)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:115: UserWarning: Output "custom_variational_layer_2" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "custom_variational_layer_2" during training.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 64, 64, 1)     0                                            
____________________________________________________________________________________________________
conv2d_6 (Conv2D)                (None, 64, 64, 1)     5           input_2[0][0]                    
____________________________________________________________________________________________________
conv2d_7 (Conv2D)                (None, 32, 32, 64)    320         conv2d_6[0][0]                   
____________________________________________________________________________________________________
conv2d_8 (Conv2D)                (None, 32, 32, 64)    36928       conv2d_7[0][0]                   
___________________________________________________________________________________________

ResourceExhaustedError: OOM when allocating tensor with shape[64]
	 [[Node: conv2d_8/bias/Assign = Assign[T=DT_FLOAT, _class=["loc:@conv2d_8/bias"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](conv2d_8/bias, conv2d_2/Const)]]

Caused by op 'conv2d_8/bias/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python3.5/dist-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop,